# Combining flats

There is one step in combining flats that is different than for most other image combination: the flats should be scaled to a common value before combining them. This is particularly important if the flats are twilight flats in which the average image value typically changes significantly as the images are being taken.

Flats are typically grouped by filter when combining them. That is, one combined flat is produced for each filter in which flats were taken.

Combination will be done for each of the two examples in the previous notebook.

In [ ]:
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt

import ccdproc as ccdp
from astropy.stats import mad_std
from astropy.visualization import hist

from convenience_functions import show_image

## Example 1

We begin by setting up an image collection for the reduced data.

In [ ]:
calibrated_path = Path('example1-reduced')

ifc = ccdp.ImageFileCollection(calibrated_path)

We'll first check what filters are present.

In [ ]:
flat_filters = set(h['filter'] for h in ifc.headers(imagetyp='flatfield'))
flat_filters

These flats are dome flats, essentially pictures of a screen in the dome illuminated by a light source, so one would not expect there to be much variable in the typical pixel value between different exposures. There is typically *some* variation, though, so we graph it below.

In [ ]:
median_count = [np.median(data) for data in ifc.data(imagetyp='flatfield')]
mean_count = [np.mean(data) for data in ifc.data(imagetyp='flatfield')]
plt.plot(median_count, label='median')
plt.plot(mean_count, label='mean')
plt.xlabel('Image number')
plt.ylabel('Count (ADU)')
plt.title('Pixel value in calibrated flat frames')
plt.legend()
print(median_count)

Although this is less frame-to-frame variation than we will see in Example 2, it is about 5%. If we were to combine these without scaling the flats to a common value then the images with higher counts would effectively get more weight than the images. 

There is a substantial difference between the mean and median of this data. Typically it is better to use the median because extreme values do not affect the median as much as the mean.

To scale the frames so that they have the same median value we need to define a function that can calculate the inverse of the median given the data.

In [ ]:
def inv_median(a):
    return 1 / np.median(a)

This function is passed into the `scale` argument of `combine` below. One combined flat is created for each filter in the data.

In [ ]:
for filt in flat_filters:
    to_combine = ifc.files_filtered(imagetyp='flatfield', filter=filt, include_path=True)
    combined_flat = ccdp.combine(to_combine,
                                 method='average', scale=inv_median,
                                 sigma_clip=True, sigma_clip_low_thresh=5, sigma_clip_high_thresh=5,
                                 sigma_clip_func=np.ma.median, signma_clip_dev_func=mad_std,
                                 mem_limit=350e6
                                )

    combined_flat.meta['combined'] = True
    dark_file_name = 'combined_flat_filter_{}.fit'.format(filt.replace("''", "p"))
    combined_flat.write(calibrated_path / dark_file_name)

In [ ]:
show_image(combined_flat, cmap='gray')

In [ ]:
to_combine = ifc.files_filtered(imagetyp='flatfield', filter=filt,
                                include_path=True)
bad_combined_flat = ccdp.combine(to_combine,
                             method='average',
                             sigma_clip=True, sigma_clip_low_thresh=5, sigma_clip_high_thresh=5,
                             sigma_clip_func=np.ma.median, signma_clip_dev_func=mad_std,
                             mem_limit=350e6
                            )

bad_combined_flat.meta['combined'] = True


In [ ]:
show_image(bad_combined_flat, cmap='gray')

## Example 2

In [ ]:
calibrated_path = Path('example2-reduced')

ifc = ccdp.ImageFileCollection(calibrated_path)

In [ ]:
flat_filters = set(h['filter'] for h in ifc.headers(imagetyp='flat'))
flat_filters

In [ ]:
median_count = [np.median(data) for data in ifc.data(imagetyp='flat')]
mean_count = [np.mean(data) for data in ifc.data(imagetyp='flat')]
plt.plot(median_count, label='median')
plt.plot(mean_count, label='mean')
plt.xlabel('Image number')
plt.ylabel('Count (ADU)')
plt.title('Pixel value in calibrated flat frames')
plt.legend()
print(median_count)

In [ ]:
def inv_median(a):
    return 1 / np.median(a)

for filt in flat_filters:
    to_combine = ifc.files_filtered(imagetyp='flat', filter=filt, include_path=True)
    combined_flat = ccdp.combine(to_combine,
                                 method='average', scale=inv_median,
                                 sigma_clip=True, sigma_clip_low_thresh=5, sigma_clip_high_thresh=5,
                                 sigma_clip_func=np.ma.median, signma_clip_dev_func=mad_std,
                                 mem_limit=350e6
                                )

    combined_flat.meta['combined'] = True
    dark_file_name = 'combined_flat_filter_{}.fit'.format(filt.replace("''", "p"))
    combined_flat.write(calibrated_path / dark_file_name)

In [ ]:
show_image(combined_flat, cmap='gray')

In [ ]:
to_combine = ifc.files_filtered(imagetyp='flat', filter=filt,
                                include_path=True)
bad_combined_flat = ccdp.combine(to_combine,
                             method='average',
                             sigma_clip=True, sigma_clip_low_thresh=5, sigma_clip_high_thresh=5,
                             sigma_clip_func=np.ma.median, signma_clip_dev_func=mad_std,
                             mem_limit=350e6
                            )

bad_combined_flat.meta['combined'] = True

In [ ]:
show_image(bad_combined_flat, cmap='gray')